In [36]:
import org.apache.spark.sql.functions._
import org.apache.spark.SparkContext._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions.{col, udf}
import org.apache.spark.ml.linalg.Vector

In [2]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}

In [3]:
val spark = SparkSession.builder().appName("test").config("spark.master", "yarn").getOrCreate()

spark = org.apache.spark.sql.SparkSession@3afa6d33


org.apache.spark.sql.SparkSession@3afa6d33

In [4]:
val train = spark.read
                    .option("delimiter", ",")
                    .option("header", "True")
                    .csv("/labs/slaba05/lab05_train.csv")
val test = spark.read
                    .option("delimiter", ",")
                    .option("header", "True")
                    .csv("/labs/slaba05/lab05_test.csv")

train = [_c0: string, ID: string ... 115 more fields]
test = [_c0: string, ID: string ... 114 more fields]


[_c0: string, ID: string ... 114 more fields]

In [7]:
train.show(1)

+------+------+--------------+------------------+------------------------+--------------------+-----------------+----------------+----------------------+-----------------------+-----------------------+------------------+---------------------+-----------------------+-------------------+--------------+-----------------+--------------------+-----------------------+--------------+------------------+--------------------+---------------+----------------+-----------------------+---------------------------+----------------------+------------------+-----------------+-------------------+-------------------+-------------+-----------------------+-----------------------+-----------------+---------------+-------------------+---------------+---------------+----------------+-------+---------------+-----------------------+-----------------+-----------+-------------------+------------------+-------------+---+-----------------------+--------------+-----------------------+--------------------+------------

In [5]:
var train1 = train.select("ID", "TARGET", "TRANS_CNT_TENDENCY3M", "TURNOVER_DYNAMIC_CC_3M", "TURNOVER_DYNAMIC_CUR_3M",
                            "TURNOVER_DYNAMIC_IL_3M", "TURNOVER_DYNAMIC_PAYM_3M",
                           "SUM_TRAN_SUP_TENDENCY3M", "SUM_TRAN_MED_TENDENCY3M", "SUM_TRAN_CLO_TENDENCY3M", "SUM_TRAN_AUT_TENDENCY3M",
                           "REST_DYNAMIC_SAVE_3M", "REST_DYNAMIC_PAYM_3M", "REST_DYNAMIC_IL_3M", "REST_DYNAMIC_FDEP_3M",
                           "REST_DYNAMIC_CUR_3M", "REST_DYNAMIC_CC_3M",
                           "CNT_TRAN_SUP_TENDENCY3M", "CNT_TRAN_MED_TENDENCY3M", "CNT_TRAN_CLO_TENDENCY3M", "CNT_TRAN_AUT_TENDENCY3M",
                           "CNT_TRAN_ATM_TENDENCY3M")
var test1 = test.select("ID",  "TRANS_CNT_TENDENCY3M", "TURNOVER_DYNAMIC_CC_3M", "TURNOVER_DYNAMIC_CUR_3M",
                        "TURNOVER_DYNAMIC_IL_3M", "TURNOVER_DYNAMIC_PAYM_3M",
                       "SUM_TRAN_SUP_TENDENCY3M", "SUM_TRAN_MED_TENDENCY3M", "SUM_TRAN_CLO_TENDENCY3M", "SUM_TRAN_AUT_TENDENCY3M",
                       "REST_DYNAMIC_SAVE_3M", "REST_DYNAMIC_PAYM_3M", "REST_DYNAMIC_IL_3M", "REST_DYNAMIC_FDEP_3M",
                       "REST_DYNAMIC_CUR_3M", "REST_DYNAMIC_CC_3M",
                       "CNT_TRAN_SUP_TENDENCY3M", "CNT_TRAN_MED_TENDENCY3M", "CNT_TRAN_CLO_TENDENCY3M", "CNT_TRAN_AUT_TENDENCY3M",
                       "CNT_TRAN_ATM_TENDENCY3M")

train1 = [ID: string, TARGET: string ... 20 more fields]
test1 = [ID: string, TRANS_CNT_TENDENCY3M: string ... 19 more fields]


[ID: string, TRANS_CNT_TENDENCY3M: string ... 19 more fields]

In [6]:
var feature_list = Array[String]()
for (t <- train1.columns if (t!="ID"))
{ train1 = train1.withColumn(t, col(t).cast(FloatType))
        feature_list +:= t}
val b = feature_list.filter(! _.contains("TARGET"))

feature_list = Array(CNT_TRAN_ATM_TENDENCY3M, CNT_TRAN_AUT_TENDENCY3M, CNT_TRAN_CLO_TENDENCY3M, CNT_TRAN_MED_TENDENCY3M, CNT_TRAN_SUP_TENDENCY3M, REST_DYNAMIC_CC_3M, REST_DYNAMIC_CUR_3M, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_IL_3M, REST_DYNAMIC_PAYM_3M, REST_DYNAMIC_SAVE_3M, SUM_TRAN_AUT_TENDENCY3M, SUM_TRAN_CLO_TENDENCY3M, SUM_TRAN_MED_TENDENCY3M, SUM_TRAN_SUP_TENDENCY3M, TURNOVER_DYNAMIC_PAYM_3M, TURNOVER_DYNAMIC_IL_3M, TURNOVER_DYNAMIC_CUR_3M, TURNOVER_DYNAMIC_CC_3M, TRANS_CNT_TENDENCY3M, TARGET)
b = Array(CNT_TRAN_ATM_TENDENCY3M, CNT_TRAN_AUT_TENDENCY3M, CNT_TRAN_CLO_TENDENCY3M, CNT_TRAN_MED_TENDENCY3M, CNT_TRAN_SUP_TENDENCY3M, REST_DYNAMIC_CC_3M, REST_DYNAMIC_CUR_3M, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_IL_3M, REST_DYNAMIC_PAYM_3M, REST_DYNAMIC_SAVE_3M, SUM...


Array(CNT_TRAN_ATM_TENDENCY3M, CNT_TRAN_AUT_TENDENCY3M, CNT_TRAN_CLO_TENDENCY3M, CNT_TRAN_MED_TENDENCY3M, CNT_TRAN_SUP_TENDENCY3M, REST_DYNAMIC_CC_3M, REST_DYNAMIC_CUR_3M, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_IL_3M, REST_DYNAMIC_PAYM_3M, REST_DYNAMIC_SAVE_3M, SUM...

In [7]:
for (t <- test1.columns if (t!="ID"))
{ test1 = test1.withColumn(t, col(t).cast(FloatType))}

In [9]:
test1 = test1.na.fill(value=0.0)
train1 = train1.na.fill(value=0.0)

test1 = [ID: string, TRANS_CNT_TENDENCY3M: float ... 19 more fields]
train1 = [ID: string, TARGET: float ... 20 more fields]


[ID: string, TARGET: float ... 20 more fields]

In [11]:
val vector = new VectorAssembler()
                    .setInputCols(b)
                    .setOutputCol("features")

vector = vecAssembler_33871a58ed39


vecAssembler_33871a58ed39

In [12]:
train1 = train1.withColumn("TARGET", col("TARGET").cast(IntegerType))

train1 = [ID: string, TARGET: int ... 20 more fields]


[ID: string, TARGET: int ... 20 more fields]

In [13]:
train1.schema

StructType(StructField(ID,StringType,true), StructField(TARGET,IntegerType,true), StructField(TRANS_CNT_TENDENCY3M,FloatType,false), StructField(TURNOVER_DYNAMIC_CC_3M,FloatType,false), StructField(TURNOVER_DYNAMIC_CUR_3M,FloatType,false), StructField(TURNOVER_DYNAMIC_IL_3M,FloatType,false), StructField(TURNOVER_DYNAMIC_PAYM_3M,FloatType,false), StructField(SUM_TRAN_SUP_TENDENCY3M,FloatType,false), StructField(SUM_TRAN_MED_TENDENCY3M,FloatType,false), StructField(SUM_TRAN_CLO_TENDENCY3M,FloatType,false), StructField(SUM_TRAN_AUT_TENDENCY3M,FloatType,false), StructField(REST_DYNAMIC_SAVE_3M,FloatType,false), StructField(REST_DYNAMIC_PAYM_3M,FloatType,false), StructField(REST_DYNAMIC_IL_3M,FloatType,false), StructField(REST_DYNAMIC_FDEP_3M,Fl...

In [14]:
val train2=vector.transform(train1)

train2 = [ID: string, TARGET: int ... 21 more fields]


[ID: string, TARGET: int ... 21 more fields]

In [15]:
val evaluator  = new BinaryClassificationEvaluator()
                        .setLabelCol("TARGET")
                        .setRawPredictionCol("probability")
                        .setMetricName("areaUnderROC")

evaluator = binEval_4135c72ebc19


binEval_4135c72ebc19

In [17]:
val clf = new RandomForestClassifier()
      .setFeaturesCol("features")
      .setLabelCol("TARGET")
      .setNumTrees(30)
      .setMaxDepth(10)
      .setSubsamplingRate(0.7)

clf = rfc_515161b09aad


rfc_515161b09aad

In [18]:
val test2=vector.transform(test1)

test2 = [ID: string, TRANS_CNT_TENDENCY3M: float ... 20 more fields]


[ID: string, TRANS_CNT_TENDENCY3M: float ... 20 more fields]

In [21]:
val model_clf = clf.fit(train2)

model_clf = RandomForestClassificationModel (uid=rfc_515161b09aad) with 30 trees


RandomForestClassificationModel (uid=rfc_515161b09aad) with 30 trees

In [22]:
val predictions_train = model_clf.transform(train2)

predictions_train = [ID: string, TARGET: int ... 24 more fields]


[ID: string, TARGET: int ... 24 more fields]

In [23]:
val predictions_test = model_rf.transform(test2)

predictions_test = [ID: string, TRANS_CNT_TENDENCY3M: float ... 23 more fields]


[ID: string, TRANS_CNT_TENDENCY3M: float ... 23 more fields]

In [37]:
val vecToArray = udf((v: Vector) =>  v.toArray)

vecToArray = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,ArrayType(DoubleType,false),Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [38]:
predictions_test.select(vecToArray(col("probability")).getItem(1).alias("target")).show(1)

+------------------+
|            target|
+------------------+
|0.2007249596167553|
+------------------+
only showing top 1 row



In [33]:
predictions_test.select(vecToArray(col("probability"))).show(1, false)

Name: org.apache.spark.sql.AnalysisException
Message: cannot resolve 'UDF(probability)' due to data type mismatch: argument 1 requires vector type, however, '`probability`' is of struct<type:tinyint,size:int,indices:array<int>,values:array<double>> type.;;
'Project [UDF(probability#2754) AS UDF(probability)#3275]
+- Project [ID#255, TRANS_CNT_TENDENCY3M#1600, TURNOVER_DYNAMIC_CC_3M#1601, TURNOVER_DYNAMIC_CUR_3M#1602, TURNOVER_DYNAMIC_IL_3M#1603, TURNOVER_DYNAMIC_PAYM_3M#1604, SUM_TRAN_SUP_TENDENCY3M#1605, SUM_TRAN_MED_TENDENCY3M#1606, SUM_TRAN_CLO_TENDENCY3M#1607, SUM_TRAN_AUT_TENDENCY3M#1608, REST_DYNAMIC_SAVE_3M#1609, REST_DYNAMIC_PAYM_3M#1610, REST_DYNAMIC_IL_3M#1611, REST_DYNAMIC_FDEP_3M#1612, REST_DYNAMIC_CUR_3M#1613, REST_DYNAMIC_CC_3M#1614, CNT_TRAN_SUP_TENDENCY3M#1615, CNT_TRAN_MED_TENDENCY3M#1616, CNT_TRAN_CLO_TENDENCY3M#1617, CNT_TRAN_AUT_TENDENCY3M#1618, CNT_TRAN_ATM_TENDENCY3M#1619, features#1793, rawPrediction#2730, probability#2754, UDF(rawPrediction#2730) AS prediction#2

In [44]:
val to_file = predictions_test.select(col("ID").alias("id"), vecToArray(col("probability")).getItem(1).alias("target")).collect()

to_file = Array([519130,0.2007249596167553], [234045,0.06381715893734766], [401256,0.07182391270292456], [551070,0.15865179793281564], [367285,0.013664677600691516], [497998,0.08363759180546831], [413082,0.08059976826784031], [349893,0.013664677600691516], [346337,0.139691797947495], [289979,0.037484759366276536], [510818,0.1203775663431398], [235935,0.0904143079451547], [532135,0.12638674327457303], [564760,0.021319244701049445], [277391,0.19298065799539518], [336830,0.1679945699987123], [356053,0.059061488084387624], [293302,0.0833953970766497], [322368,0.019201960714992708], [406041,0.06930372550682254], [569179,0.08017685274563383], [191405,0.052434616653255516], [489011,0.0378207769369385], [265952,0.08070009572956452], [193718,0.053931763089018696]...


Array([519130,0.2007249596167553], [234045,0.06381715893734766], [401256,0.07182391270292456], [551070,0.15865179793281564], [367285,0.013664677600691516], [497998,0.08363759180546831], [413082,0.08059976826784031], [349893,0.013664677600691516], [346337,0.139691797947495], [289979,0.037484759366276536], [510818,0.1203775663431398], [235935,0.0904143079451547], [532135,0.12638674327457303], [564760,0.021319244701049445], [277391,0.19298065799539518], [336830,0.1679945699987123], [356053,0.059061488084387624], [293302,0.0833953970766497], [322368,0.019201960714992708], [406041,0.06930372550682254], [569179,0.08017685274563383], [191405,0.052434616653255516], [489011,0.0378207769369385], [265952,0.08070009572956452], [193718,0.053931763089018696]...

In [91]:
predictions_test.select("probability").show(1, false)

+----------------------------------------+
|probability                             |
+----------------------------------------+
|[0.7980660019990831,0.20193399800091705]|
+----------------------------------------+
only showing top 1 row



In [41]:
to_file.show()

+------+--------------------+
|    id|              target|
+------+--------------------+
|519130|  0.2007249596167553|
|234045| 0.06381715893734766|
|401256| 0.07182391270292456|
|551070| 0.15865179793281564|
|367285|0.013664677600691516|
|497998| 0.08363759180546831|
|413082| 0.08059976826784031|
|349893|0.013664677600691516|
|346337|   0.139691797947495|
|289979|0.037484759366276536|
|510818|  0.1203775663431398|
|235935|  0.0904143079451547|
|532135| 0.12638674327457303|
|564760|0.021319244701049445|
|277391| 0.19298065799539518|
|336830|  0.1679945699987123|
|356053|0.059061488084387624|
|293302|  0.0833953970766497|
|322368|0.019201960714992708|
|406041| 0.06930372550682254|
+------+--------------------+
only showing top 20 rows



In [45]:
val string_to_write = "id\ttarget" + "\n" + to_file.map(x => Array(x(0), x(1))
                                                    ).map(y => y.mkString("\t")).mkString("\n")

string_to_write = 


id	target
519130	0.2007249596167553
234045	0.06381715893734766
401256	0.07182391270292456
551070	0.15865179793281564
367285	0.013664677600691516
497998	0.08363759180546831
413082	0.08059976826784031
349893	0.013664677600691516
346337	0.139691797947495
289979	0.037484759366276536
510818	0.1203775663431398
235935	0.0904143079451547
532135	0.12638674327457303
564760	0.021319244701049445
277391	0.19298065799539518
336830	0.1679945699987123
356053	0.059061488084387624
293302	0.0833953970766497
322368	0.019201960714992708
406041	0.06930372550682254
569179	0.08017685274563383
191405	0.052434616653255516
489011	0.0378207769369385
265952	0.08070009572956452
193718	0.053931763089018696
241961	0.1704272764669898
522250	0.05090881267922347
486617	0.053047851743765244
56303...


In [48]:
import java.io.PrintWriter
new PrintWriter("lab05.csv") { write(string_to_write); close }

$anon$1@6f020ddf